Importing the required libraries and constants

In [4]:
import os, pathlib, json, numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import f1_score

SEED = 42
IMG_SIZE = (224, 224)
BATCH = 32

ROOT_DIR = pathlib.Path("data/")
TRAIN_DIR = ROOT_DIR / "training"
VAL_DIR   = ROOT_DIR / "validation"
TEST_DIR  = ROOT_DIR / "testing"

TRAIN_SPLIT = 0.8
VAL_SPLIT   = 0.1
TEST_SPLIT  = 0.1

MODEL_DIR = "models"

Distribute the images to create training, validation and testing sets

In [2]:
# import shutil
# from collections import defaultdict

# np.random.seed(SEED )
# # Gather all images by class from all splits
# all_images = defaultdict(list)
# for split_dir in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
#     for class_dir in split_dir.iterdir():
#         if class_dir.is_dir():
#             for img_file in class_dir.glob("*.jpg"):
#                 all_images[class_dir.name].append(img_file)
#             for img_file in class_dir.glob("*.png"):
#                 all_images[class_dir.name].append(img_file)
#             for img_file in class_dir.glob("*.jpeg"):
#                 all_images[class_dir.name].append(img_file)

# # Redistribute according to split ratios
# for class_name, img_files in all_images.items():
#     img_files = np.array(img_files)
#     np.random.shuffle(img_files)
#     n_total = len(img_files)
#     n_train = int(n_total * TRAIN_SPLIT)
#     n_val = int(n_total * VAL_SPLIT)
#     n_test = n_total - n_train - n_val

#     splits = [
#         (TRAIN_DIR / class_name, img_files[:n_train]),
#         (VAL_DIR / class_name, img_files[n_train:n_train+n_val]),
#         (TEST_DIR / class_name, img_files[n_train+n_val:]),
#     ]

#     for target_dir, files in splits:
#         target_dir.mkdir(parents=True, exist_ok=True)
#         for f in files:
#             dest = target_dir / f.name
#             if f.resolve() != dest.resolve():
#                 shutil.move(str(f), str(dest))

# print("Data redistribution complete.")
# # Print out number of images in each split for each class
# for class_name in all_images.keys():
#     train_count = len(list((TRAIN_DIR / class_name).glob("*.jpg"))) + len(list((TRAIN_DIR / class_name).glob("*.png"))) + len(list((TRAIN_DIR / class_name).glob("*.jpeg")))
#     val_count = len(list((VAL_DIR / class_name).glob("*.jpg"))) + len(list((VAL_DIR / class_name).glob("*.png"))) + len(list((VAL_DIR / class_name).glob("*.jpeg")))
#     test_count = len(list((TEST_DIR / class_name).glob("*.jpg"))) + len(list((TEST_DIR / class_name).glob("*.png"))) + len(list((TEST_DIR / class_name).glob("*.jpeg")))
#     print(f"Class '{class_name}': Training={train_count}, Validation={val_count}, Testing={test_count}")

This file is for training the various image classification models

In [6]:
import tensorflow as tf
from tensorflow import keras

# Build datasets WITHOUT validation_split
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH,
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR,
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH,
    shuffle=False   # keep order stable for metrics/plots
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH,
    shuffle=False   # keep order stable for metrics/plots
)

# Class names (e.g., ['01-minor','02-moderate','03-severe'])
class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print("Classes:", class_names)

# (Optional) pretty labels for display, stripping the numeric prefixes
pretty_names = [c.split('-', 1)[-1] if '-' in c else c for c in class_names]

AUTOTUNE = tf.data.AUTOTUNE
def configure(ds, training=False):
    ds = ds.cache()
    if training:
        ds = ds.shuffle(1024, seed=SEED)
    return ds.prefetch(AUTOTUNE)

train_ds = configure(train_ds, training=True)
val_ds   = configure(val_ds)
test_ds  = configure(test_ds)

Found 2035 files belonging to 4 classes.
Found 253 files belonging to 4 classes.
Found 258 files belonging to 4 classes.
Classes: ['00-none', '01-minor', '02-moderate', '03-severe']


Setting up F1 function for performance evaluation

In [8]:
import numpy as np
from sklearn.metrics import f1_score

class MacroF1Callback(keras.callbacks.Callback):
    def __init__(self, val_ds, num_classes):
        super().__init__()
        self.val_ds = val_ds
        self.num_classes = num_classes

    def on_epoch_end(self, epoch, logs=None):
        y_true = []
        y_pred = []
        for xb, yb in self.val_ds:
            probs = self.model.predict(xb, verbose=0)
            y_pred.extend(np.argmax(probs, axis=1))
            # yb may be ints or one-hot depending on your pipeline:
            if yb.ndim == 2:  # one-hot
                y_true.extend(np.argmax(yb.numpy(), axis=1))
            else:             # integer labels
                y_true.extend(yb.numpy())
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        macro_f1 = f1_score(y_true, y_pred, average="macro")
        logs = logs or {}
        logs["val_macro_f1"] = macro_f1
        print(f"\nval_macro_f1: {macro_f1:.4f}")

In [9]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.08),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
], name="augment")

preprocess = keras.applications.mobilenet_v2.preprocess_input

base = keras.applications.mobilenet_v2.MobileNetV2(
    include_top=False, weights="imagenet", input_shape=(*IMG_SIZE, 3)
)
base.trainable = False

inputs = keras.Input(shape=(*IMG_SIZE, 3))
x = data_augmentation(inputs)
x = preprocess(x)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.summary()



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ augment (Sequential)            │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │         5,124 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,263,108 (8.63 MB)

 Trainable params: 5,124 (20.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

# 5. Training with Checkpoint

Early stopping with checkpoint. we start with frozen and then unlock the model. GPU enabled runtime will run much faster. If you plot the curves what can it tell you on the epochs?

In [6]:
EPOCHS_FROZEN = 30
EPOCHS_FINE   = 60

model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")]
)

# If your datasets are integer labels, map to one-hot for this loss:
train_oh = train_ds.map(lambda x, y: (x, tf.one_hot(y, NUM_CLASSES)))
val_oh   = val_ds.map(lambda x, y: (x, tf.one_hot(y, NUM_CLASSES)))

f1_cb = MacroF1Callback(val_ds, NUM_CLASSES)

#Model filename
filename = MODEL_DIR + "\\mobilenetv2_frozen.keras"

ckpt1 = keras.callbacks.ModelCheckpoint(
    filename, monitor="val_macro_f1", mode="max",
    save_best_only=True, verbose=1
)
early = keras.callbacks.EarlyStopping(monitor="val_macro_f1", mode="max",
                                      patience=5, restore_best_weights=True)
reduce = keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                           factor=0.5, patience=2, verbose=1)

hist1 = model.fit(
    train_oh, validation_data=val_oh,
    epochs=EPOCHS_FROZEN, callbacks=[f1_cb, ckpt1, early, reduce], verbose=2
)

Epoch 1/30

val_macro_f1: 0.4978

Epoch 1: val_macro_f1 improved from None to 0.49778, saving model to models\mobilenetv2_frozen.keras
64/64 - 32s - 495ms/step - acc: 0.3278 - loss: 1.5224 - val_acc: 0.5257 - val_loss: 1.2120 - val_macro_f1: 0.4978 - learning_rate: 3.0000e-04
Epoch 2/30

val_macro_f1: 0.5921

Epoch 2: val_macro_f1 improved from 0.49778 to 0.59206, saving model to models\mobilenetv2_frozen.keras
64/64 - 19s - 295ms/step - acc: 0.4855 - loss: 1.2380 - val_acc: 0.6047 - val_loss: 1.0832 - val_macro_f1: 0.5921 - learning_rate: 3.0000e-04
Epoch 3/30

val_macro_f1: 0.6030

Epoch 3: val_macro_f1 improved from 0.59206 to 0.60301, saving model to models\mobilenetv2_frozen.keras
64/64 - 19s - 298ms/step - acc: 0.5233 - loss: 1.1366 - val_acc: 0.6166 - val_loss: 0.9939 - val_macro_f1: 0.6030 - learning_rate: 3.0000e-04
Epoch 4/30

val_macro_f1: 0.6263

Epoch 4: val_macro_f1 improved from 0.60301 to 0.62626, saving model to models\mobilenetv2_frozen.keras
64/64 - 19s - 300ms/step 

6. Unfreeze (Fine Tuning)

In [7]:
# Unfreeze top layers except BatchNorm
for layer in base.layers[-60:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")]
)

filename = MODEL_DIR + "\\mobilenetv2_finetuned.keras"
f1_cb2 = MacroF1Callback(val_ds, NUM_CLASSES)
ckpt2 = keras.callbacks.ModelCheckpoint(
    filename, monitor="val_macro_f1", mode="max",
    save_best_only=True, verbose=1
)

hist2 = model.fit(
    train_oh, validation_data=val_oh,
    epochs=EPOCHS_FINE, callbacks=[f1_cb2, ckpt2, early, reduce], verbose=2
)

Epoch 1/60

val_macro_f1: 0.6645

Epoch 1: val_macro_f1 improved from None to 0.66449, saving model to models\mobilenetv2_finetuned.keras
64/64 - 36s - 564ms/step - acc: 0.6467 - loss: 0.8839 - val_acc: 0.6680 - val_loss: 0.8809 - val_macro_f1: 0.6645 - learning_rate: 1.0000e-05
Epoch 2/60

val_macro_f1: 0.6793

Epoch 2: val_macro_f1 improved from 0.66449 to 0.67931, saving model to models\mobilenetv2_finetuned.keras
64/64 - 23s - 362ms/step - acc: 0.6830 - loss: 0.8256 - val_acc: 0.6798 - val_loss: 0.8624 - val_macro_f1: 0.6793 - learning_rate: 1.0000e-05
Epoch 3/60

val_macro_f1: 0.6846

Epoch 3: val_macro_f1 improved from 0.67931 to 0.68457, saving model to models\mobilenetv2_finetuned.keras
64/64 - 23s - 365ms/step - acc: 0.6919 - loss: 0.8184 - val_acc: 0.6877 - val_loss: 0.8336 - val_macro_f1: 0.6846 - learning_rate: 1.0000e-05
Epoch 4/60

val_macro_f1: 0.6803

Epoch 4: val_macro_f1 did not improve from 0.68457
64/64 - 23s - 353ms/step - acc: 0.7219 - loss: 0.7803 - val_acc: 0.68

7. Performance Evaluation
there are also other things to show. can you think of any?

In [8]:
from sklearn.metrics import classification_report, confusion_matrix
MODEL_FILE = MODEL_DIR + "\\mobilenetv2_finetuned.keras"

best = keras.models.load_model(MODEL_FILE, compile=False)
best.compile(optimizer=keras.optimizers.Adam(1e-5),
             loss=keras.losses.CategoricalCrossentropy(),
             metrics=["accuracy"])

y_true, y_pred = [], []
for xb, yb in test_ds:
    probs = best.predict(xb, verbose=0)
    y_pred.extend(np.argmax(probs, axis=1))
    if yb.ndim == 2: y_true.extend(np.argmax(yb.numpy(), axis=1))
    else:            y_true.extend(yb.numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)

print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification report:")
print(classification_report(y_true, y_pred, target_names=class_names))
print("Macro-F1:", f1_score(y_true, y_pred, average="macro"))


Confusion matrix:
 [[64  2  3  2]
 [ 2 41  7  0]
 [ 9 11 27 20]
 [ 4  4 13 49]]

Classification report:
              precision    recall  f1-score   support

     00-none       0.81      0.90      0.85        71
    01-minor       0.71      0.82      0.76        50
 02-moderate       0.54      0.40      0.46        67
   03-severe       0.69      0.70      0.70        70

    accuracy                           0.70       258
   macro avg       0.69      0.71      0.69       258
weighted avg       0.69      0.70      0.69       258

Macro-F1: 0.6922916287809905


In [11]:
import matplotlib.pyplot as plt
import random
MODEL_FILE = MODEL_DIR + "\\mobilenetv2_finetuned.keras"

# Create a dataset with raw images and labels (no one-hot)
val_raw = tf.keras.utils.image_dataset_from_directory(
    ROOT_DIR,
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH,
    validation_split=0.1,
    subset="validation",
    shuffle=True
)

# Get a small batch
images, labels = next(iter(val_raw))
pred_probs = best.predict(images)
pred_ids = np.argmax(pred_probs, axis=1)
true_ids = labels.numpy()

# Plot 12 random samples
plt.figure(figsize=(12, 9))
for i, idx in enumerate(random.sample(range(len(images)), 12)):
    ax = plt.subplot(3, 4, i + 1)
    img = images[idx].numpy().astype("uint8")
    true_label = class_names[true_ids[idx]]
    pred_label = class_names[pred_ids[idx]]
    color = "green" if pred_label == true_label else "red"
    plt.imshow(img)
    plt.title(f"Pred: {pred_label}\nTrue: {true_label}", color=color, fontsize=9)
    plt.axis("off")
plt.tight_layout()
plt.show()

Found 7626 files belonging to 6 classes.
Using 762 files for validation.


NameError: name 'best' is not defined